In [18]:
# In the first cell of your Model_Training.ipynb notebook

# ====================================================================
# IMPORTANT: CHANGE THIS VARIABLE TO TRAIN FOR DIFFERENT PRODUCTS
# ====================================================================
PRODUCT_NAME = 'zipper'  # <-- CHANGE THIS (e.g., 'hazelnut', 'cable', 'screw')
# ====================================================================


import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model

# --- Configuration ---
DATA_DIR = f'datasets/{PRODUCT_NAME}/train/good/'
MODEL_SAVE_PATH = f'models/{PRODUCT_NAME}_model.h5'
IMG_SIZE = (128, 128)
EPOCHS = 50
BATCH_SIZE = 16

print(f"--- Starting Training for Product: {PRODUCT_NAME} ---")

# --- 1. Load and Preprocess Data ---
if not os.path.exists(DATA_DIR):
    print(f"FATAL ERROR: Dataset directory not found at '{DATA_DIR}'. Please check the product name and folder structure.")
else:
    print("Loading training images from:", DATA_DIR)
    good_images = []
    for img_name in os.listdir(DATA_DIR):
        img_path = os.path.join(DATA_DIR, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, IMG_SIZE)
            good_images.append(img)

    if not good_images:
        print(f"FATAL ERROR: No images found in '{DATA_DIR}'.")
    else:
        good_images = np.array(good_images).astype('float32') / 255.0
        good_images = np.reshape(good_images, (len(good_images), IMG_SIZE[0], IMG_SIZE[1], 1))
        print(f"Data loaded successfully: {len(good_images)} images.")

        # --- 2. Build the Autoencoder Model ---
        print("Building the model architecture...")
        input_img = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 1))
        x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
        encoded = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
        x = UpSampling2D((2, 2))(x)
        decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
        autoencoder = Model(input_img, decoded)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')
        autoencoder.summary()

        # --- 3. Train the Model ---
        print("\nStarting model training...")
        autoencoder.fit(good_images, good_images, epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=True, validation_split=0.2)

        # --- 4. Save the Final Model ---
        os.makedirs('models', exist_ok=True)
        print(f"\nTraining complete. Saving model to '{MODEL_SAVE_PATH}'...")
        autoencoder.save(MODEL_SAVE_PATH)
        print("Model saved successfully!")
        print(f"--- Finished Training for {PRODUCT_NAME} ---")

--- Starting Training for Product: zipper ---
Loading training images from: datasets/zipper/train/good/
Data loaded successfully: 240 images.
Building the model architecture...


Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_15 (InputLayer)          │ (None, 128, 128, 1)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_75 (Conv2D)                   │ (None, 128, 128, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_30 (MaxPooling2D)      │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_76 (Conv2D)                   │ (None, 64, 64, 16)          │           4,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_31 (MaxPooling2D)      │ (None, 32, 32, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_77 (Conv2D)                   │ (None, 32, 32, 16)          │           2,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_30 (UpSampling2D)      │ (None, 64, 64, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_78 (Conv2D)                   │ (None, 64, 64, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ up_sampling2d_31 (UpSampling2D)      │ (None, 128, 128, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_79 (Conv2D)                   │ (None, 128, 128, 1)         │             289 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12,193 (47.63 KB)

 Trainable params: 12,193 (47.63 KB)

 Non-trainable params: 0 (0.00 B)


Starting model training...
Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 100ms/step - loss: 0.1108 - val_loss: 0.0872
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - loss: 0.0685 - val_loss: 0.0107
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 0.0125 - val_loss: 0.0072
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 79ms/step - loss: 0.0076 - val_loss: 0.0060
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - loss: 0.0058 - val_loss: 0.0049
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - loss: 0.0049 - val_loss: 0.0043
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 0.0044 - val_loss: 0.0040
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - loss: 0.0040 - val_loss: 0.0036
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - loss: 0.0037 - val_loss: 0.0034
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 80ms/step - loss: 0.0034 - val_loss: 0.0032
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - loss: 0.0032 - val_loss: 0.0030
Epoch 12/50
12/12 ━━━━━━━━━━━━━━━━


Training complete. Saving model to 'models/zipper_model.h5'...
Model saved successfully!
--- Finished Training for zipper ---
